In [19]:
import numpy as np
from pyqubo import Binary
import pyqubo
import neal
import matplotlib.pyplot as plt 
from sympy import *
from datetime import datetime
import pandas as pd


## Input Data 

In [20]:
#input data 


#Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].to_numpy()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].to_numpy()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].to_numpy()

n_assets = len(asset_quantity)

#Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].to_numpy()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].to_numpy()

n_accounts = len(account_exposure)

#single limits
# single_limits = pd.read_csv("data/sample_single_limits.csv", header=None).to_numpy() 

#haircuts 
haircuts = pd.read_csv("data/sample_haircuts.csv", header=None).to_numpy()/100

#Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

# Penalty Terms

In [41]:

#Penalty Terms
lambda_cost_fn = 1e5
lambda_consistency = 1
lambda_exposure = 1
# lambda_single_limits = 1


# Defining the QUBO

In [42]:
# Binary variables

n_binary = 7
max_binary_value = 2**(n_binary) - 1 

binary_coeff_vector = np.zeros(n_binary)

for i in range(n_binary):
    binary_coeff_vector[i] = 2**i


#Decision variable

binary_variables = []

for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            globals()["x" + str(i + 1) + str(j + 1) + str(k+1)] = Binary("x" + str(i + 1) + str(j + 1) + str(k+1))
            binary_variables.append(globals()["x" + str(i + 1) + str(j + 1) + str(k + 1)])


#Slack variable for consistency 
num_slack_consistency_arr = np.zeros(shape=(n_assets))

for i in range(n_assets):
    num_slack_consistency_arr[i] = int(np.ceil(np.log2(max_binary_value)))


consistency_slack_variables_arr = [] 

for count, i in enumerate(num_slack_consistency_arr): 
    consistency_slack_variables_arr.append(pyqubo.Array.create("consistency_asset" + str(count + 1) + "_slack", shape = int(i), vartype = "BINARY"))

    
# slack variables for single limit

#num_slack_single_limit_arr = np.zeros(shape=(n_assets, n_accounts))

# for i in range(n_assets):
#     for j in range(n_accounts):
#         #print((single_limits[i][j]/asset_quantity[i])*max_binary_value)
#         try:
#             num_slack_single_limit_arr[i][j] = int(np.ceil(np.log2((single_limits[i][j]/asset_quantity[i])*max_binary_value)))
#         except OverflowError:
#             print("pass")

# single_limit_slack_variables_arr = []  #going to be a triple nested array   

# for asset_counter, i in enumerate(num_slack_single_limit_arr):
#     aux_single_limit_slack_arr = []
#     for account_counter, j in enumerate(i):
#         if j == 0:
#             aux_single_limit_slack_arr.append([])
#         else:
#             aux_single_limit_slack_arr.append(pyqubo.Array.create("single_lim_asset" + str(asset_counter + 1) + "_account" + str(account_counter + 1) + "_slack", shape = int(j), vartype = "BINARY"))
#     single_limit_slack_variables_arr.append(aux_single_limit_slack_arr)


# Constructing the QUBO equation 

This involves the QUBO objective function, the consistency contraints, the exposure constraints.

In [43]:

#QUBO objective function 

cost_fn = 0

for i in range(n_assets):
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        cost_fn += aux_term1*cost_factor_matrix[i][j]

cost_fn /= max_binary_value


In [44]:
# Consistency constraint 

consistency_penalty_term = 0

#slack part
consistency_penalty_slack = []

for i in consistency_slack_variables_arr:
    aux_term = 0
    for exponent,j in enumerate(i):
        aux_term += (2**exponent)*j
    consistency_penalty_slack.append(aux_term)


#decision variable part
for i in range(n_assets):
    aux_term2 = 0
    for j in range(n_accounts):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1)
    consistency_penalty_term += (aux_term2 - max_binary_value + consistency_penalty_slack[i])**2

In [45]:
#Exposure constraint

exposure_penalty_term = 0

for j in range(n_accounts):
    aux_term2 = 0
    for i in range(n_assets):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
        aux_term2 += (aux_term1*asset_quantity[i]*asset_value[i]*haircuts[i][j])/max_binary_value
    exposure_penalty_term += (aux_term2 - account_exposure[j]) **2 

exposure_penalty_term /= np.mean(asset_quantity*asset_value)

In [46]:
# Single limit constraint

# single_limit_penalty_term = 0

# #slack part

# single_limit_penalty_slack = [] 

# for i in range(n_assets):
#     aux_array = []
#     for j in range(n_accounts):
#         aux_term = 0
#         if single_limit_slack_variables_arr[i][j] == []:
#             aux_term += 0
#         else:    
#             for exponent,k in enumerate(single_limit_slack_variables_arr[i][j]):
#                 aux_term += (2**exponent)*k  
#         aux_array.append(aux_term)
#     single_limit_penalty_slack.append(aux_array)

# #decision variable part

# for i in range(n_assets):
#     for j in range(n_accounts):
#         aux_term2 = 0
#         aux_term1 = 0
#         for k in range(n_binary):
#             aux_term1 += binary_coeff_vector[k]* globals()["x" + str(i + 1) + str(j + 1) + str(k+1)]
#         aux_term2 += aux_term1 - (single_limits[i][j]/asset_quantity[i])*max_binary_value + single_limit_penalty_slack[i][j]
#         #print(aux_term2**2)
#         single_limit_penalty_term += aux_term2**2




In [47]:


#Final QUBO
QUBO_equation =lambda_cost_fn*cost_fn + lambda_consistency*consistency_penalty_term  + lambda_exposure*exposure_penalty_term #+ lambda_single_limits*single_limit_penalty_term

#compile model
model = QUBO_equation.compile()
qubo, offset = model.to_qubo()


# Run Solver

In [48]:
#Run Solver

bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()
num_reads = 100
num_sweeps = 1000
sampleset = sa.sample(bqm, num_reads=num_reads, num_sweeps=num_sweeps)

#Get best sample
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
results = best_sample.sample
pprint(f"The minimum energy is: {best_sample.energy}")


The minimum energy is: 50180.52714473568


# Verify Results

In [49]:

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        decimal = 0
        for k in range(n_binary):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]
    allocation_solution[i] = asset


=====Each Individual Allocation Percentage=====
Assign 0.00% of asset 1 to account 1
Assign 0.00% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 0.00% of asset 1 to account 5
Assign 11.81% of asset 2 to account 1
Assign 24.41% of asset 2 to account 2
Assign 0.00% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 0.00% of asset 2 to account 5
Assign 0.00% of asset 3 to account 1
Assign 0.00% of asset 3 to account 2
Assign 0.00% of asset 3 to account 3
Assign 0.00% of asset 3 to account 4
Assign 0.00% of asset 3 to account 5
Assign 0.00% of asset 4 to account 1
Assign 0.00% of asset 4 to account 2
Assign 0.00% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 2.36% of asset 5 to account 1
Assign 8.66% of asset 5 to account 2
Assign 2.36% of asset 5 to account 3
Assign 1.57% of asset 5 to account 4
Assign 2.36% of asset 5 to account 5
Assign 24.41% of asset 6 

In [50]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")  

====Validation for Consistency====
0.00% of asset 1 posted
36.22% of asset 2 posted
0.00% of asset 3 posted
0.00% of asset 4 posted
17.32% of asset 5 posted
57.48% of asset 6 posted
0.00% of asset 7 posted
24.41% of asset 8 posted
0.00% of asset 9 posted
0.00% of asset 10 posted


In [51]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]:,} required") 
    print(f"{(exposure_solution[i]/account_exposure[i] - 1) *100.:0.2f}%")

=====Validation for Exposure======
$819,129.8067239374 of collateral posted, $824,000.0 required
-0.59%
$657,290.178172119 of collateral posted, $669,292.0 required
-1.79%
$542,073.7930437214 of collateral posted, $548,600.0 required
-1.19%
$60,494.17437111193 of collateral posted, $64,844.52 required
-6.71%
$455,883.28018683975 of collateral posted, $472,000.0 required
-3.41%


In [52]:
#print("===Validation of Single limits =====")
# for i in range(n_assets):
#     for j in range(n_accounts):
#         decimal = 0
#         for k in range(n_binary):
#             decimal +=(2**k)*my_bit_array[i][j][k]
#         allocation = (decimal/max_binary_value)*asset_quantity[i] 
#         print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
#         print(f"Constraint Satisfied?: {allocation <= single_limits[i][j]}")
        

# Export to csv

In [53]:
#Saving to an array
Q_value_balanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        decimal = 0
        for k in range(n_binary):
            decimal +=binary_coeff_vector[k]*results.get("x" + str(i + 1) + str(j + 1) + str(k+1))
        Q_value_balanced_py_qubo[i][j] = decimal/max_binary_value

pprint(f"Objective value: {sum(sum(Q_value_balanced_py_qubo*cost_factor_matrix))}")

Objective value: 0.49527559055118114


In [ ]:
#Uncomment line below to save file to csv

#pd.DataFrame(Q_value_balanced_py_qubo).to_csv("Q_value_balanced_py_qubo_no_single_lim1.csv", header=None, index=None)
